In [1]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, GRPO_WRITING_CONFIG, SMOL_LM_135M

cfg = GRPO_WRITING_CONFIG
cfg.train_batch_size = 2
cfg.num_generations = 2
cfg.model_id_or_path = SMOL_LM_135M
cfg.max_samples = 10

/home/brianf/smolmodels/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-13 19:16:14,240	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
wrapper = TrainerWrapper(cfg, True)

2025-04-13 19:16:19.013 | INFO     | trl_wrapper.trainer_wrapper:__init__:297 - Setting padding side to: right


In [7]:
wrapper.init_model()

2025-04-13 19:16:39.732 | INFO     | trl_wrapper.trainer_wrapper:init_model:312 - Using device: cuda:0
2025-04-13 19:16:39.733 | INFO     | trl_wrapper.trainer_wrapper:init_model:326 - Loading model HuggingFaceTB/SmolLM2-135M-Instruct with attn_impl: flash_attention_2


In [14]:
wrapper.init_data_module()

2025-04-13 19:20:22.237 | INFO     | trl_wrapper.trainer_wrapper:init_data_module:352 - Using chat template override: smollmv2
Map: 100%|██████████| 1149/1149 [00:00<00:00, 1274.73 examples/s]


In [15]:
wrapper.data_module.train_dataset

Dataset({
    features: ['prompt'],
    num_rows: 10336
})

In [12]:
wrapper.init_trainer()

2025-04-13 19:17:12.337 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:390 - Saving output to: ./runs/04-13-19-17-382863-smollm2-135m-instruct-
2025-04-13 19:17:12.339 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:413 - Initializing trainer, run_name: 04-13-19-17-382863-smollm2-135m-instruct-, wandb project: writing-grpo
2025-04-13 19:17:12.340 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:496 - Using vllm: False


In [17]:
wrapper.train()

2025-04-13 19:20:51.694 | INFO     | trl_wrapper.trainer_wrapper:train:659 - Starting training.
2025-04-13 19:21:19.909 | INFO     | synthetic_data.tasks.writing:score_writing:411 - Judging 2 completions with gemini-2.0-flash


kwargs: {}
scores: [{'Adherence to Instructions': 2.0, 'Believable Character Actions': 3.0, 'Nuanced Characters': 1.0, 'Consistent Voice/Tone of Writing': 5.0, 'Imagery and Descriptive Quality': 3.0, 'Elegant Prose': 2.0, 'Emotionally Engaging': 1.0, 'Emotionally Complex': 1.0, 'Coherent': 2.0, 'Meandering': 3.0, 'Weak Dialogue': 1.0, "Tell-Don't-Show": 5.0, 'Unsurprising or Uncreative': 1.0, 'Amateurish': 2.0, 'Purple Prose': 1.0, 'Overwrought': 1.0, 'Incongruent Ending Positivity': 1.0, 'Unearned Transformations': 1.0, 'Well-earned Lightness or Darkness': 1.0, 'Sentences Flow Naturally': 1.0, 'Overall Reader Engagement': 1.0, 'Overall Impression': 1.0}, {'Adherence to Instructions': 0.0, 'Believable Character Actions': 0.0, 'Nuanced Characters': 0.0, 'Consistent Voice/Tone of Writing': 0.0, 'Imagery and Descriptive Quality': 0.0, 'Elegant Prose': 0.0, 'Emotionally Engaging': 0.0, 'Emotionally Complex': 0.0, 'Coherent': 0.0, 'Meandering': 0.0, 'Weak Dialogue': 0.0, "Tell-Don't-Show": 

KeyboardInterrupt: 

In [6]:
wrapper.data_module.reward_functions()[0](["hi"], ["hi"])

2025-04-13 19:16:31.439 | INFO     | synthetic_data.tasks.writing:score_writing:411 - Judging 1 completions with gemini-2.0-flash


kwargs: {}


[60.0]